In [ ]:
!pip install transformers

In [29]:
"""
Image captioning
"""
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 100
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict(image):
  images=[image]
  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [ ]:
import requests

from google.colab import drive
drive.mount('/content/drive')

from PIL import Image

import os 

ty=input("Please, enter the name of your LoRa : ").replace(" ","_")

os.makedirs("/content/drive/MyDrive/Loras/"+ty+"/dataset/", exist_ok=True)

for page in range(0,10):
  search_url="https://librex.pufe.org/api.php?q="+ty+"&t=1&p="+str(page*10)#we are using the LibreX API to search for pictures (You can choose another instance from https://github.com/hnhx/librex/)
  print(search_url)
  results=requests.get(search_url)

  for n,img in enumerate(results.json()):
    img_url=img["thumbnail"]
    image=Image.open(requests.get(img_url, stream=True).raw)
    raw_image = image.convert('RGB')

    print(img_url)
    image.save("/content/drive/MyDrive/Loras/"+ty+"/dataset/"+str(n+49*page)+".png","PNG")

    file = open("/content/drive/MyDrive/Loras/"+ty+"/dataset/"+str(n+49*page)+".txt", "w")
    r=predict(image)#image captioning
    print(r)
    file.write(ty.replace("_"," ")+", "+r[0])#["generated_text"])
    file.close()
